In [1]:
!gdown 1ZkmwcjtK1NE_E6BQsuOtx-8d_8r_N0fP

Downloading...
From (original): https://drive.google.com/uc?id=1ZkmwcjtK1NE_E6BQsuOtx-8d_8r_N0fP
From (redirected): https://drive.google.com/uc?id=1ZkmwcjtK1NE_E6BQsuOtx-8d_8r_N0fP&confirm=t&uuid=1c96b504-1aa1-4c11-9546-4f65a3d2d1b4
To: /content/model.keras
100% 28.8M/28.8M [00:00<00:00, 64.7MB/s]


In [2]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from IPython.display import Image
import cv2
from keras.models import load_model
import numpy as np
import PIL
import io


In [3]:
# Define helper functions
def js_to_image(js_reply):
    """
    Params:
    js_reply: JavaScript object containing image from webcam
    Returns:
    img: OpenCV BGR image
    """
    # decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

In [4]:
# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
    """
    Params:
    bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
    Returns:
    bytes: Base64 image byte string
    """
    # convert array into PIL image
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # format bbox into png for return
    bbox_PIL.save(iobuf, format='png')
    # format return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
    return bbox_bytes

In [5]:
def video_stream():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
    stream.getVideoTracks()[0].stop();
    video.remove();
    div.remove();

    video = null;
    div = null;
    stream = null;
    imgElement = null;
    captureCanvas = null;
    labelElement = null;
    }

    function onAnimationFrame() {
    if (!shutdown) {
    window.requestAnimationFrame(onAnimationFrame);
    }
    if (pendingResolve) {
    var result = "";
    if (!shutdown) {
    captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
    result = captureCanvas.toDataURL('image/jpeg', 0.8)
    }
    var lp = pendingResolve;
    pendingResolve = null;
    lp(result);
    }
    }

    async function createDom() {
    if (div !== null) {
    return stream;
    }
    div = document.createElement('div');
    div.style.border = '2px solid black';
    div.style.padding = '3px';
    div.style.width = '100%';
    div.style.maxWidth = '600px';
    document.body.appendChild(div);

    const modelOut = document.createElement('div');
    modelOut.innerHTML = "<span>Status:</span>";
    labelElement = document.createElement('span');
    labelElement.innerText = 'No data';
    labelElement.style.fontWeight = 'bold';
    modelOut.appendChild(labelElement);
    div.appendChild(modelOut);

    video = document.createElement('video');
    video.style.display = 'block';
    video.width = div.clientWidth - 6;
    video.setAttribute('playsinline', '');
    video.onclick = () => { shutdown = true; };
    stream = await navigator.mediaDevices.getUserMedia (
    {video: { facingMode: "environment"}});
    div.appendChild(video);
    imgElement = document.createElement('img');
    imgElement.style.position = 'absolute';
    imgElement.style.zIndex = 1;
    imgElement.onclick = () => { shutdown = true; };
    div.appendChild(imgElement);

    const instruction = document.createElement('div');
    instruction.innerHTML =
    '<span style="color: red; font-weight: bold;">' +
    'When finished, click here or on the video to stop this demo</span>';
    div.appendChild(instruction);
    instruction.onclick = () => { shutdown = true; };

    video.srcObject = stream;
    await video.play();
    captureCanvas = document.createElement('canvas');
    captureCanvas.width = 640; //video.videoWidth;
    captureCanvas.height = 480; //video.videoHeight;
    window.requestAnimationFrame(onAnimationFrame);

    return stream;
    }
    async function stream_frame(label, imgData) {
    if (shutdown) {
    removeDom();
    shutdown = false;
    return '';
    }
    var preCreate = Date.now();
    stream = await createDom();

    var preShow = Date.now();
    if (label != "") {
    labelElement.innerHTML = label;
    }

    if (imgData != "") {
    var videoRect = video.getClientRects()[0];
    imgElement.style.top = videoRect.top + "px";
    imgElement.style.left = videoRect.left + "px";
    imgElement.style.width = videoRect.width + "px";
    imgElement.style.height = videoRect.height + "px";
    imgElement.src = imgData;
    }

    var preCapture = Date.now();
    var result = await new Promise(function(resolve, reject) {
    pendingResolve = resolve;
    });
    shutdown = false;

    return {'create': preShow - preCreate,
    'show': preCapture - preShow,
    'capture': Date.now() - preCapture,
    'img': result};
    }
    ''')
    display(js)


In [6]:
def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

In [7]:
# Load the Haarcascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect face and preprocess image
def detect_face(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        roi_gray = roi_gray.reshape(1, 48, 48, 1) / 255.0
        return roi_gray
    else:
        return None

In [10]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialize bounding box to empty
bbox = ''
count = 0

# Load the trained model
model = load_model("/content/model.keras")
IMG_SIZE = 48
EMOTIONS = ['happiness', 'sadness', 'neutral']

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])
    # Detect face and preprocess image
    face = detect_face(frame)
    if face is not None:
        pred = model.predict(face, verbose=0)
        # Get the predicted emotion label
        pred_label = EMOTIONS[np.argmax(pred)]
        # Update label_html with the predicted emotion
        label_html = f'{pred_label}'
    else:
        label_html = 'No face detected'

<IPython.core.display.Javascript object>